<a href="https://colab.research.google.com/github/sherryfengshopify/ai-gym/blob/main/ReAct_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install openai

In [ ]:
# @title Import and setup
from google.cloud import bigquery
from google.colab import data_table
import bigframes.pandas as bpd
import pandas as pd
from openai import OpenAI
import json
import yaml

# project = 'sdp-prd-build-ecosystem' # Project ID inserted based on the query results selected to explore
# location = 'us-central1' # Location inserted based on the query results selected to explore

llm = OpenAI(
    base_url="https://openai-proxy.shopify.ai/v1",
    api_key="shopify-eyJpZCI6IjhhOTFmMmIyYjU2MzQ3NzFlMzQyZWU2NDRjMzFmYTY3IiwibW9kZSI6InBlcnNvbmFsIiwiZW1haWwiOiJzaGVycnkuZmVuZ0BzaG9waWZ5LmNvbSIsImV4cGlyeSI6MTczMzQyODAzNX0=-PJz6nHOe6gFKKQzAWyX8ySg5WzQ6llYcfcTFSKJiOJM="
)

# client = bigquery.Client(project=project, location='US')
# data_table.enable_dataframe_formatter()

In [ ]:
from google.colab import auth
auth.authenticate_user()

In [ ]:
import openai
import re
import httpx
import os

In [ ]:
# client = OpenAI()

MODEL = 'gpt-4o-mini'
prompt = 'Write something short but funny.'

chat_completion = llm.chat.completions.create(
    model=MODEL,
    messages=[
        {
            'role': 'user',
            'content': prompt,
        }
    ]
)

In [ ]:
chat_completion.choices[0].message.content

'Why did the scarecrow win an award? Because he was outstanding in his field!'

In [ ]:
# @title Creating the agent class

class Agent:
  def __init__(self, system=''):
    self.system = system
    self.messages = []

    if self.system:
      self.messages.append({'role': 'system', 'content': system})

  def __call__(self, prompt):
    self.messages.append({'role': 'user', 'content': prompt})
    result = self.execute()
    self.messages.append({'role': 'assistant', 'content': result})
    return result

  def execute(self, model='gpt-4o-mini', temperature=0):
    completion = llm.chat.completions.create(
        model=model,
        temperature=temperature,
        messages=self.messages
    )
    return completion.choices[0].message.content

In [ ]:
# @title Creating the ReAct prompt
prompt = '''
You run in a loop of Thought, Action, PAUSE, Observation.
At the end of the loop you output an Answer
Use Thought to describe your thoughts about the question you have been asked.
Use Action to run one of the actions available to you - then return PAUSE.
Observation will be the result of running those actions.

Your available actions are:

calculate:
e.g. calculate 4 * 7 / 3
Runs a calculation and returns the number - uses Python so be sure to use floating point syntax if necessary

get_cost:
e.g. get_cost: book
returns the cost of a book

wikipedia:
e.g. wikipedia: LangChain
Returns a summary from searching Wikipedia

Always look things up on Wikipedia if you have the opportunity to do so.

Example session #1:

Question: How much does a pen cost?
Thought: I should look the pen cost using get_cost
Action: get_cost: pen
PAUSE

You will be called again with this:

Observation: A pen costs $5


Example session #2:

Question: What is the capital of France?
Thought: I should look up France on Wikipedia
Action: wikipedia: France
PAUSE

You will be called again with this:

Observation: France is a country. The capital is Paris.

You then output:

Answer: The capital of France is Paris.

'''.strip()

In [ ]:
# @title Creating the tools
